Start notebook

In [1]:
import random

# Set the global seed
_GLOBAL_SEED = 42
random.seed(_GLOBAL_SEED)

In [2]:
import pandas as pd

# Read the training dataset
train_df = pd.read_csv('../_Dataset/train_dataset.csv')

# Read the test dataset
test_df = pd.read_csv('../_Dataset/test_dataset.csv')


In [3]:
train_genetic_disorder_df = train_df.drop("disorder_subclass", axis=1)
train_disorder_subclass_df = train_df

test_disorder_subclass_df = test_df.drop("genetic_disorder", axis=1)

Format the dataset

In [4]:
train_genetic_disorder_x = train_genetic_disorder_df.drop("genetic_disorder",axis=1)
train_genetic_disorder_y = train_genetic_disorder_df["genetic_disorder"]

train_disorder_subclass_x = train_disorder_subclass_df.drop("disorder_subclass",axis=1)
train_disorder_subclass_y = train_disorder_subclass_df["disorder_subclass"]

test_disorder_subclass_x = test_disorder_subclass_df.drop("disorder_subclass",axis=1)
test_disorder_subclass_y = test_disorder_subclass_df["disorder_subclass"]

Scale the Genetic Disorder training data

In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler and transform the training data
train_genetic_disorder_x = scaler.fit_transform(train_genetic_disorder_x)

Train the best model for identifing the genetic disorder

In [6]:
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier

catb_classifier = CatBoostClassifier(random_state=_GLOBAL_SEED, verbose=False)

# Assuming catb_classifier is your CatBoostClassifier
catb_classifier = CalibratedClassifierCV(catb_classifier, method='sigmoid', cv=5)

# Train the classifier
catb_classifier.fit(train_genetic_disorder_x, train_genetic_disorder_y)

CalibratedClassifierCV(cv=5,
                       estimator=<catboost.core.CatBoostClassifier object at 0x000001E8C5EBF950>)

Predict the genetic disorder for the test dataset using the genetic disorder classfier

In [7]:
test_genetic_disorder_predictions = catb_classifier.predict(test_disorder_subclass_x)

test_disorder_subclass_x['genetic_disorder'] = test_genetic_disorder_predictions

In [8]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler and transform the training data
train_disorder_subclass_x = scaler.fit_transform(train_disorder_subclass_x)

# Use the same scaler to transform the test data
test_disorder_subclass_x = scaler.transform(test_disorder_subclass_x)

Declare the models

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV

logistic_regression_classifier = LogisticRegression(solver='newton-cg', random_state=_GLOBAL_SEED)
kn_classifier = KNeighborsClassifier()
decision_tree_classifier = DecisionTreeClassifier(random_state=_GLOBAL_SEED)
gaussian_nb_classifier = GaussianNB()
random_forest_classifier = RandomForestClassifier(random_state=_GLOBAL_SEED)
gradient_boosting_classifier = GradientBoostingClassifier(random_state=_GLOBAL_SEED)
xgb_classifier = XGBClassifier(random_state=_GLOBAL_SEED)
lgb_classifier = LGBMClassifier(random_state=_GLOBAL_SEED)
svc_classifier = SVC(decision_function_shape='ovo')
catb_classifier = CatBoostClassifier(random_state=_GLOBAL_SEED, verbose=False)

# Assuming catb_classifier is your CatBoostClassifier
catb_classifier = CalibratedClassifierCV(catb_classifier, method='sigmoid', cv=5)

Train the models on the genetic disorder training dataset

In [16]:
logistic_regression_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
kn_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
decision_tree_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
gaussian_nb_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
random_forest_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
gradient_boosting_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
xgb_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
lgb_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
svc_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)
catb_classifier.fit(train_disorder_subclass_x, train_disorder_subclass_y)

models_collection = {
    'Logistic Regression': logistic_regression_classifier, 
    'K-Nearest Neighbors': kn_classifier, 
    'Decision Tree': decision_tree_classifier, 
    'Gaussian Naive Bayes': gaussian_nb_classifier, 
    'Random Forest': random_forest_classifier, 
    'Gradient Boosting': gradient_boosting_classifier, 
    'XGBoost': xgb_classifier, 
    'LightGBM': lgb_classifier,
    'Support Vector Classifier': svc_classifier,
    'CatBoost': catb_classifier
}

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001313 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 724
[LightGBM] [Info] Number of data points in the train set: 14437, number of used features: 30
[LightGBM] [Info] Start training from score -4.886202
[LightGBM] [Info] Start training from score -5.287090
[LightGBM] [Info] Start training from score -1.741575
[LightGBM] [Info] Start training from score -2.375633
[LightGBM] [Info] Start training from score -2.699223
[LightGBM] [Info] Start training from score -3.439823
[LightGBM] [Info] Start training from score -1.350441
[LightGBM] [Info] Start training from score -1.522392
[LightGBM] [Info] Start training from score -1.948060
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

In [17]:
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error

# Initialize an empty dictionary to store the results
results = {}

for model_name, model in models_collection.items():
    # Make predictions on the test data
    predictions = model.predict(test_disorder_subclass_x)
    
    # Calculate accuracy
    accuracy = accuracy_score(test_disorder_subclass_y, predictions)
    
    # Calculate recall
    recall = recall_score(test_disorder_subclass_y, predictions, average='weighted')
    
    # Calculate mean squared error
    mse = mean_squared_error(test_disorder_subclass_y, predictions)
    
    # Store the results
    results[model_name] = {'Accuracy': accuracy, 'Recall': recall, 'Mean Squared Error': mse}
    
# Sort the results by accuracy
sorted_results = sorted(results.items(), key=lambda item: item[1]['Accuracy'], reverse=True)


In [18]:
# Print the sorted results
for model_name, metrics in sorted_results:
    print("---------------------------")
    print(f"Model: {model_name}")
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value}")
    print("---------------------------")

---------------------------
Model: Gradient Boosting
Accuracy: 0.2897506925207756
Recall: 0.2897506925207756
Mean Squared Error: 5.665096952908587
---------------------------
---------------------------
Model: CatBoost
Accuracy: 0.2883656509695291
Recall: 0.2883656509695291
Mean Squared Error: 5.984764542936288
---------------------------
---------------------------
Model: XGBoost
Accuracy: 0.28753462603878116
Recall: 0.28753462603878116
Mean Squared Error: 5.712188365650969
---------------------------
---------------------------
Model: LightGBM
Accuracy: 0.2867036011080332
Recall: 0.2867036011080332
Mean Squared Error: 6.002493074792244
---------------------------
---------------------------
Model: Random Forest
Accuracy: 0.2739612188365651
Recall: 0.2739612188365651
Mean Squared Error: 6.424376731301939
---------------------------
---------------------------
Model: K-Nearest Neighbors
Accuracy: 0.26204986149584486
Recall: 0.26204986149584486
Mean Squared Error: 8.148476454293629
----

Use the top 4 models to create an ensemle model

In [19]:
from sklearn.ensemble import VotingClassifier

# Get the top 4 models
top_4_models = sorted_results[:4]

# Create a list of tuples where each tuple contains a model name and an estimator
estimators = [(model_name, models_collection[model_name]) for model_name, _ in top_4_models]

# Create the ensemble model
ensemble = VotingClassifier(estimators=estimators, voting='soft')

# Train the ensemble model
ensemble.fit(train_disorder_subclass_x, train_disorder_subclass_y)

# Make predictions on the test data
ensemble_predictions = ensemble.predict(test_disorder_subclass_x)

# Calculate and print the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(test_disorder_subclass_y, ensemble_predictions)
print(f"Ensemble accuracy: {ensemble_accuracy}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 724
[LightGBM] [Info] Number of data points in the train set: 14437, number of used features: 30
[LightGBM] [Info] Start training from score -4.886202
[LightGBM] [Info] Start training from score -5.287090
[LightGBM] [Info] Start training from score -1.741575
[LightGBM] [Info] Start training from score -2.375633
[LightGBM] [Info] Start training from score -2.699223
[LightGBM] [Info] Start training from score -3.439823
[LightGBM] [Info] Start training from score -1.350441
[LightGBM] [Info] Start training from score -1.522392
[LightGBM] [Info] Start training from score -1.948060
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[